In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

import env

In [ ]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/curriculum_logs'
df = pd.read_sql('SELECT * FROM logs JOIN cohorts on logs.user_id = cohorts.id', url)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#change the dtypes to the correct types

In [ ]:
df.user_id = df.user_id.astype('object')

In [ ]:
df.cohort_id = df.cohort_id.astype('object')

In [ ]:
df.id = df.id.astype('object')

In [ ]:
df.program_id = df.program_id.astype('object')

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [ ]:
df.path.value_counts(ascending = False)

A: It looks like lesson jacascript-i is the most referenced lesson with 1486 requests

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


In [ ]:
# I want to concatenate the time and date column
df['date'] = df['date'] + ' ' + df['time']

In [ ]:
df.head()

In [ ]:
df = df.drop(columns='time')

In [ ]:
# change the df to a datetime index
df.date = pd.to_datetime(df.date)
df = df.set_index('date')

In [ ]:
df.head()

In [ ]:
# detect anomalies: start by gettting the count
df.path.value_counts()

In [ ]:
# next find the frequency in the form of a percentage
df.path.value_counts(normalize=True).tail(50)

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [ ]:
# create a dataframe
path_df = value_counts_and_frequencies(df.path)
path_df.head(25)

In [ ]:
# check for missing values
df.isna().sum()

In [ ]:
#remove missing values
df = df[~df['cohort_id'].isnull()]

In [ ]:
df.isna().sum()

In [ ]:
#dropping the column deleted_at since I won't need that for anything
df = df.drop(columns='deleted_at')

In [ ]:
df.shape

In [ ]:
# conditional probabilities --- cohort given path
cohort_id_given_path = (
    df.groupby('path')
    .cohort_id.value_counts(normalize=True)
    .rename('proba_cohort_id_given_path')
    .reset_index()
)
cohort_id_given_path.head(30)

In [ ]:
# there are 1436 paths 

In [ ]:
# look at classification/knn and class/log reg as possible anomalies
# there are many more that are accessed by cohort 28-staff only 

In [ ]:
df[df.path == 'classification/knn']

- possible anomalies:

do these lessons (class/knn and class/log-reg no longer exist is that why it was accessed solely by cohort 28?


In [ ]:
# going back to exploring each feature individually

In [ ]:
# detect anomalies start by gettting the count
df.ip.value_counts()

In [ ]:
# next find the frequency in the form of a percentage
df.ip.value_counts(normalize=True)

In [ ]:
# this tells me with ip address 97.105.19.61 doing .379 of the requests it is probably
# the Code up ip address

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [ ]:
# create a dataframe
ip_df = value_counts_and_frequencies(df.ip)
ip_df.head(20)

In [ ]:
# conditional probabilities
cohort_id_given_ip = (
    df.groupby('ip')
    .cohort_id.value_counts(normalize=True)
    .rename('proba_cohort_id_given_ip')
    .reset_index()
)
cohort_id_given_ip.head()

In [ ]:
# looking for anomalies

cohort_id_given_ip[cohort_id_given_ip.proba_cohort_id_given_ip < 1].head(30)

In [ ]:
# let's look more into cohort_id

In [ ]:
# detect anomalies start by gettting the count
df.cohort_id.value_counts()

In [ ]:
# next find the frequency in the form of a percentage
df.cohort_id.value_counts(normalize=True)

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [ ]:
# create a dataframe
cohort_id_df = value_counts_and_frequencies(df.cohort_id)
cohort_id_df

In [ ]:
# conditional probabilities
cohort_id_given_start_date = (
    df.groupby('start_date')
    .cohort_id.value_counts(normalize=True)
    .rename('proba_cohort_id_start_date')
    .reset_index()
)
cohort_id_given_start_date.head()

there seems to be an anomaly either line 3 or 4 because there can't be 100%  chance that the start date for cohort 22 was both June and August of 2014

In [ ]:
# let's look more at start_date individually

In [ ]:
# detect anomalies start by gettting the count
df.start_date.value_counts(ascending = False)

In [ ]:
# next find the frequency in the form of a percentage
df.start_date.value_counts(normalize=True)

In [ ]:
# create a dataframe
start_date_df = value_counts_and_frequencies(df.start_date)
start_date_df.head()

In [ ]:
# conditional probabilities
start_date_given_ip = (
    df.groupby('ip')
    .start_date.value_counts(normalize=True)
    .rename('proba_start_date_given_ip')
    .reset_index()
)
start_date_given_ip

In [ ]:
# looking for anomalies
start_date_given_ip[start_date_given_ip.proba_start_date_given_ip < 1]

for these ip addresses with more than one start date any ip location other than 
97.105.19.61 which I determined was the Code up location may be anomalies, so let's look at just those other addresses

In [ ]:
df.ip[df.ip != '97.105.19.61'].head(30)

In [ ]:
start_date_given_ip[start_date_given_ip.ip != '97.105.19.61']

In [ ]:
# if I want to answer which cohorts are accessing which paths the most

In [ ]:
df['path_1'] = df.path.str.split('/').str[0]

df['path_2'] = df.path.str.split('/').str[1]

In [ ]:
df.head(50)

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

So here I want to find the 'active' dates for students and after that note anything else that might be useful to include about these individuals.



Anomaly detection is not about a specific methodology just find it witht the Pandas I know